# Batch convert json files from Labelme to labelled images

### batch convert labels - merged classes

In [ ]:
import os
import subprocess
import shutil
import numpy as np
from PIL import Image

# Define input and output directories
# input_dir = r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Panos_clipped_higher_res"  # Directory with JSON files
input_dir = r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\RICS\RICS_wagga_sample_images"  # Directory with JSON files
# output_dir = r"C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\GSV_annotations_converted_merged"  # Base output directory
output_dir = r"D:\Wagga\RICS\annotations_converted_merged"  # Base output directory


# Define the fixed class list directly in the code
original_class_list  = ["_background_", "foundation", "partial door", "front door", "partial garage door", "stairs","garage door"]

# Define the target class list after merging
target_class_list = ["_background_", "foundation", "front door", "garage door", "stairs"]
target_class_to_id = {name: i for i, name in enumerate(target_class_list)}
# Mapping original classes to target classes
merge_mapping = {
    "partial door": "front door",
    "partial garage door": "garage door"
}
# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Process each JSON file
for file_name in os.listdir(input_dir):
    if file_name.endswith(".json"):
        image_output_path = os.path.join(images_dir, f"{file_name.replace('.json', '.png')}")
        if os.path.exists(image_output_path):
            print('file exists, skipping...')
        else:
            input_path = os.path.join(input_dir, file_name)
            temp_output_dir = os.path.join(output_dir, file_name.replace(".json", ""))
            # Run labelme_json_to_dataset
            subprocess.run([
                "labelme_json_to_dataset",
                input_path,
                "-o",
                temp_output_dir
            ])

            # Load the generated label.png
            label_path = os.path.join(temp_output_dir, "label.png")
            label_image = np.array(Image.open(label_path))

            # Read label names from label_names.txt
            label_names_path = os.path.join(temp_output_dir, "label_names.txt")
            with open(label_names_path, "r") as f:
                label_names = [line.strip() for line in f]  # Strip whitespace

            # Dynamically create a class-to-ID mapping for the current image
            class_to_id = {}
            current_id = 0
            for label_name in label_names:
                # Apply merge mapping
                target_label = merge_mapping.get(label_name, label_name)
                if target_label not in class_to_id:
                    class_to_id[target_label] = current_id
                    current_id += 1

            # Remap labels using the dynamic mapping
            remapped_label = np.zeros_like(label_image, dtype=np.uint8)
            for original_id, label_name in enumerate(label_names):
                # Apply merge mapping
                target_label = merge_mapping.get(label_name, label_name)
                target_id = target_class_to_id[target_label]
                remapped_label[label_image == original_id] = target_id

            # Save the remapped label.png
            remapped_label_path = os.path.join(labels_dir, f"{file_name.replace('.json', '.png')}")
            Image.fromarray(remapped_label).save(remapped_label_path)

            # Move the original image to the images directory
            img_path = os.path.join(temp_output_dir, "img.png")
            if os.path.exists(img_path):
                os.rename(img_path, image_output_path)

            # Cleanup: Remove temporary output directory
            for temp_file in os.listdir(temp_output_dir):
                os.remove(os.path.join(temp_output_dir, temp_file))
            os.rmdir(temp_output_dir)

print(f"Processing completed. Images saved to {images_dir}, labels saved to {labels_dir}.")

### batch convert labels - original classes

In [ ]:
# import os
# import subprocess
# import shutil
# import numpy as np
# from PIL import Image

# # Define input and output directories
# input_dir = r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Panos_clipped_higher_res"  # Directory with JSON files
# output_dir = r"C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\GSV_annotations_converted"  # Base output directory


# # Define the fixed class list directly in the code
# class_list = ["_background_", "foundation", "partial door", "front door", "partial garage door", "stairs","garage door"]
# class_to_id = {name: i for i, name in enumerate(class_list)}

# # Separate folders for images and labels
# images_dir = os.path.join(output_dir, "images")
# labels_dir = os.path.join(output_dir, "labels")

# # Ensure the output directories exist
# os.makedirs(images_dir, exist_ok=True)
# os.makedirs(labels_dir, exist_ok=True)

# # Process each JSON file in the input directory
# for file_name in os.listdir(input_dir):
#     if file_name.endswith(".json"):  # Process only JSON files
#         input_path = os.path.join(input_dir, file_name)
#         temp_output_dir = os.path.join(output_dir, file_name.replace(".json", ""))
#         os.makedirs(temp_output_dir, exist_ok=True)
#         # Run the LabelMe command to convert JSON to dataset
#         subprocess.run([
#             "labelme_json_to_dataset",
#             input_path,
#             "-o",
#             temp_output_dir
#         ])
#         # Load the generated label.png and remap labels to fixed IDs
#         label_path = os.path.join(temp_output_dir, "label.png")
#         label_image = np.array(Image.open(label_path))

#         # Create a new label image with consistent IDs
#         remapped_label = np.zeros_like(label_image, dtype=np.uint8)
#         with open(os.path.join(temp_output_dir, "label_names.txt"), "r") as f:
#             label_names = [line.strip() for line in f]

#         # Map existing labels to fixed IDs
#         for label_name in label_names:
#             if label_name in class_to_id:
#                 label_id = class_to_id[label_name]
#                 remapped_label[label_image == label_names.index(label_name)] = label_id

#         # Save the remapped label.png
#         remapped_label_path = os.path.join(labels_dir, f"{file_name.replace('.json', '.png')}")
#         Image.fromarray(remapped_label).save(remapped_label_path)

#         # Move the original image to the images directory
#         img_path = os.path.join(temp_output_dir, "img.png")
#         if os.path.exists(img_path):
#             image_output_path = os.path.join(images_dir, f"{file_name.replace('.json', '.png')}")
#             os.rename(img_path, image_output_path)

#         # Cleanup: Remove temporary output directory
#         for temp_file in os.listdir(temp_output_dir):
#             os.remove(os.path.join(temp_output_dir, temp_file))
#         os.rmdir(temp_output_dir)

# print(f"Images saved to: {images_dir}")
# print(f"Labels saved to: {labels_dir}")
